# Introduction

### ***Purpose***
Knowing how to invest personal savings is important for maximizing wealth for all: individuals, households and institutions. And it goes far beyond just locking money in a savings account. With this optimization problem we are addressing the needs to faciliate personal investment decision-making. 


### ***Method***
We consider a number of assets in which one could invest and devise a model to optimize a portfolio which generates the highest return for a given level of risk. 
We appreciate that users can have different levels of target return, time horizons and risk-averseness. 
For this purpose, we run the model for several different scenarios in which we adjust for different metrics. 

<img src="citi-bike.jpg" width="500">

In [5]:
# Import various packages
import pandas as pd
import numpy as np
import statistics
import folium # visualisation package for spatial data (plot of maps)
import seaborn as sns # general visualization package 
import matplotlib.pyplot as plt # general visualization package 
#next command allows you to display the figures in the notebook
%matplotlib inline    

In [6]:
# Import the gurobi package
import gurobipy as gp
from gurobipy import GRB,quicksum
import datetime

# Data

In [7]:
riskfree = pd.read_csv('data/riskfree.csv')
bitcoin = pd.read_csv('data/bitcoin.csv')
gold = pd.read_csv('data/Gold.csv')
ftse = pd.read_csv('data/FTSE.csv')
bank_rates = pd.read_excel('data/bank_rates.xlsx')
house_prices = pd.read_excel('data/house_prices.xlsx')

In [8]:
riskfree = riskfree.dropna()
bitcoin = bitcoin.dropna()
gold = gold.dropna()
ftse = ftse.dropna()
bank_rates = bank_rates.dropna()
house_prices = house_prices.dropna()

In [9]:
riskfree['Date'] = pd.to_datetime(riskfree['Date'], format='%b-%y')
riskfree = riskfree.sort_values(by='Date')

In [10]:
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])
bitcoin = bitcoin.sort_values(by="Date")

In [11]:
gold['Date'] = pd.to_datetime(gold['Date'])
gold = gold.sort_values(by="Date")

In [12]:
ftse['Date'] = pd.to_datetime(ftse['Date'])
ftse = ftse.sort_values(by="Date")

In [13]:
bank_rates['Rate'] = bank_rates['Rate']/12
bank_rates

,Date,Rate
0,2015-01-01,0.041667
1,2015-02-01,0.041667
2,2015-03-01,0.041667
3,2015-04-01,0.041667
4,2015-05-01,0.041667
...,...,...
91,2022-08-01,0.145833
92,2022-09-01,0.187500
93,2022-10-01,0.187500
94,2022-11-01,0.250000


In [14]:
house_prices['Date'] = pd.to_datetime(house_prices['Date'])
house_prices = house_prices.sort_values(by="Date")

In [15]:
values = pd.DataFrame()

In [16]:
# riskfree = pd.read_csv('riskfree.csv')
# bitcoin = pd.read_csv('bitcoin.csv')
# gold = pd.read_csv('Gold.csv')
# ftse = pd.read_csv('FTSE.csv')
# bank_rates = pd.read_excel('bank_rates.xlsx')
# house_prices = pd.read_excel('house_prices.xlsx')
bank_rates

,Date,Rate
0,2015-01-01,0.041667
1,2015-02-01,0.041667
2,2015-03-01,0.041667
3,2015-04-01,0.041667
4,2015-05-01,0.041667
...,...,...
91,2022-08-01,0.145833
92,2022-09-01,0.187500
93,2022-10-01,0.187500
94,2022-11-01,0.250000


In [17]:
values.index = riskfree['Date'].dt.strftime('%m/%Y')
values['riskfree'] = riskfree['Price'].values
values['bitcoin'] = bitcoin['Open'].values
values["gold"] = gold['Price'].values
values["ftse"] = ftse['Price'].values
values['ftse'] = values['ftse'].str.replace(',','')
values["ftse"] = values["ftse"].astype(float)
values["house_prices"] = house_prices['PX_MID'].values
values["bank_rates"] = bank_rates['Rate'].values

In [18]:
values

,riskfree,bitcoin,gold,ftse,house_prices,bank_rates
Date,,,,,,
01/2015,1.341,320.434998,441.71,6749.40,190665,0.041667
02/2015,1.789,216.867004,418.89,6946.66,190827,0.041667
03/2015,1.579,254.283005,408.56,6773.04,191537,0.041667
04/2015,1.835,244.223007,408.29,6960.63,193225,0.041667
05/2015,1.804,235.938995,410.84,6984.43,195313,0.041667
...,...,...,...,...,...,...
08/2022,2.799,23336.718750,596.06,7284.15,292206,0.145833
09/2022,4.096,20050.498050,577.35,6893.81,294274,0.187500
10/2022,3.523,19431.105470,566.54,7094.53,294996,0.187500


In [19]:
returns = pd.DataFrame()
returns = values.pct_change(1)
returns['bank_rates'] = values['bank_rates']

In [20]:
returns

,riskfree,bitcoin,gold,ftse,house_prices,bank_rates
Date,,,,,,
01/2015,NaN,NaN,NaN,NaN,NaN,0.041667
02/2015,0.334079,-0.323211,-0.051663,0.029226,0.000850,0.041667
03/2015,-0.117384,0.172530,-0.024660,-0.024993,0.003721,0.041667
04/2015,0.162128,-0.039562,-0.000661,0.027697,0.008813,0.041667
05/2015,-0.016894,-0.033920,0.006246,0.003419,0.010806,0.041667
...,...,...,...,...,...,...
08/2022,0.510523,0.177405,-0.031206,-0.018762,0.011072,0.145833
09/2022,0.463380,-0.140818,-0.031389,-0.053588,0.007077,0.187500
10/2022,-0.139893,-0.030892,-0.018723,0.029116,0.002453,0.187500


In [21]:
mean_returns = returns.mean()
mean_returns = mean_returns.to_dict()

In [22]:
mean_returns['riskfree'] = 0.00322 # current yearly rate is 3.864%, thus monthly rate is 3.864/12
mean_returns

{'riskfree': 0.00322,
 'bitcoin': 0.06687589620239774,
 'gold': 0.004493369767621386,
 'ftse': 0.0017219801309813312,
 'house_prices': 0.004640390645573267,
 'bank_rates': 0.045876736111111115}

In [23]:
variance = returns.var()
variance = variance.to_dict()

In [24]:
variance['riskfree'] = 0 # assume
variance

{'riskfree': 0,
 'bitcoin': 0.05276732797060849,
 'gold': 0.001509312501510058,
 'ftse': 0.0013549727170874425,
 'house_prices': 0.00012109719949019301,
 'bank_rates': 0.002178543418006822}

In [25]:
covariance = values.cov()

In [26]:
covariance['riskfree'] = 0
covariance.iloc[0] = 0
covariance

,riskfree,bitcoin,gold,ftse,house_prices,bank_rates
riskfree,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
bitcoin,0,2.652161e+08,1.174646e+06,1.890792e+06,3.215854e+08,-19.239194
gold,0,1.174646e+06,9.018851e+03,-3.173816e+03,2.008619e+06,0.315971
ftse,0,1.890792e+06,-3.173816e+03,2.873414e+05,4.278158e+06,7.791421
house_prices,0,3.215854e+08,2.008619e+06,4.278158e+06,6.936991e+08,567.095727
bank_rates,0,-1.923919e+01,3.159706e-01,7.791421e+00,5.670957e+02,0.002179


## User-defined input

## Vivian's code

In [27]:
#vivan's code - linked to extract the values that were user's input from the dashboard

#our temporary input
# time_frame = 2

# amount_invested = 1000 

# min_return = 5000

# max_risk = 0.1

time_frame = 12
min_return = 5000
max_risk = 10
amount_invested = 100000

## Model creation

In [28]:
# Create a new model:
m = gp.Model("portfolio")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-09


## Data

*Define the dictionaries containing the data:*

In [29]:
# Fixed inputs
assets = ['riskfree', 'bitcoin', 'gold', 'ftse', 'house_prices', 'bank_rates'] 

# will need to get real data from Ishaan to substitute below:
returns = mean_returns

risks = variance

## Formulate the optimal allocation strategy as an integer program


1. **Decision variables:** We create a decision variable $x_a$ for the amount invested for each asset $a \in {\rm assets}$.
This means that we have the following decision variables: $x_{crypto}, x_{real estate}, x_{FTSE},x_{tech stocks}, x_{savings account}, x_{bond 3m}, x_{bond 1 yr}, x_{bond 5 yr},x_{bond 10 yr}$. The quantity of the optimal budget allocation over assets is unknown.


2. **Constraints:** We need to ensure that the total budget spent across all the keywords does not exceed the fixed amount to be invested that was entered by the user. Mathematically, this requirement is expressed using a linear constraint: 
<br><br>
$$  x_{crypto} + x_{real estate} + x_{FTSE} + x_{tech stocks} + x_{savings account} + x_{bond 3m}+x_{bond 1 yr}+x_{bond 5 yr}+x_{bond 10 yr} \leq \quad money\quad invested$$
<br>
In this constraint, the total money invested spent should not exceed the restriction of the user input.

    
3. **Objective function:** Now, we need to select the objective function. *What should we optimize for?* The goal is find a combination of money allocation for each asset that maximizes the final return. It is shown with the following expression:

<br><br>
$$  \max_{x_a}  \quad\quad\quad  x_{crypto}\cdot(1+return rate_{crypto})^{time frame} + x_{real estate}\cdot(1+return rate_{real estate})^{time frame} + x_{FTSE}\cdot(1+return rate_{FTSE})^{time frame} + x_{tech stocks}\cdot(1+return rate_{tech stocks})^{time frame} + x_{savings account}\cdot(1+return rate_{savings account})^{time frame} + x_{bond 3m}\cdot(1+return rate_{bond 3m})^{time frame} + x_{bond 1 yr}\cdot(1+return rate_{bond 1 yr})^{time frame} + x_{bond 5 yr}\cdot(1+return rate_{bond 5 yr})^{time frame} + x_{bond 10 yr}\cdot(1+return rate_{bond 10 yr})^{time frame}$$
<br>

In this function, we maximize the return for a given time horizon specified by the user (ie. *time_frame* refers to the time period for which the user is ready to have their money locked in the investment portfolio). 

Shortly, we can express the function as following:

$$ \max_{x_a} \quad\quad\quad \sum_{a=1}^{n} X_a \cdot (1+return rate_a)^{time frame}$$


## Decision variables

In [30]:
investment_amount = m.addVars(assets, vtype=GRB.INTEGER, lb = 0, name = "investment_amount")

## Constraints 

Add constraints for:

1) minimum output return that is accepted (ie. that the output should be greater than or equal to the return desired by the user)

2) maximum level of risk accepted (ie. that the output portfolio should have risk that does not exceed the maximum risk specified by the user)

In [31]:
#proxy values
# time_frame = 10
# min_return = 0.03
# max_risk = 0.1
# amount_invested = 10000

#min return accepted       
m.addConstr((quicksum(investment_amount[a]*((1+returns[a])**(12*time_frame)) for a in assets)-amount_invested >= min_return),
             name = "minimum return accepted")

#max risk accepted
# m.addConstr((quicksum(investment_amount[a1]*investment_amount[a2]*risks[a1]*risks[a2]*covariance.loc[a1,a2]/((amount_invested)**2) 
#                        for a1 in assets for a2 in assets)) <= max_risk, name="maximum risk accepted")

#max risk accepted corrected
m.addConstr((quicksum(investment_amount[a1]*investment_amount[a2]*covariance.loc[a1,a2]/((amount_invested)**2) 
                       for a1 in assets for a2 in assets)) <= (max_risk**2), name="maximum risk accepted")

#sum of investments
m.addConstr((quicksum(investment_amount[a1] for a1 in assets)) == amount_invested, name="sum of investments")

<gurobi.Constr *Awaiting Model Update*>

## Objective

Formulate the objective function 

In [32]:
# Objective function:
m.setObjective(quicksum(investment_amount[a]*((1+returns[a])**(12*time_frame)) for a in assets), 
              GRB.MAXIMIZE)

#add the objective function to minimize risk 
#send Vivian the output from running the model

## Solve

After having formulated and implemented the integer program, we can now optimize the portfolio allocation and printout the optimal return:

In [33]:
# Run the optimization
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nPortfolio Return: %g' % m.objVal)
        print('\nInvestment Amount:')
        investment_amountx = m.getAttr('x', investment_amount) 
        for a in assets:            
                print('%s %g' % (a, investment_amountx[a]))
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xdad02658
Model has 1 quadratic constraint
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [2e-13, 7e-02]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+05]
  QRHS range       [1e+02, 1e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 6 columns, 6 nonzeros
Presolved model has 1 quadratic constraint(s)
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 6.453976e+07, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

In [34]:
time_frame

12

In [35]:
risks

{'riskfree': 0,
 'bitcoin': 0.05276732797060849,
 'gold': 0.001509312501510058,
 'ftse': 0.0013549727170874425,
 'house_prices': 0.00012109719949019301,
 'bank_rates': 0.002178543418006822}

In [36]:
returns

{'riskfree': 0.00322,
 'bitcoin': 0.06687589620239774,
 'gold': 0.004493369767621386,
 'ftse': 0.0017219801309813312,
 'house_prices': 0.004640390645573267,
 'bank_rates': 0.045876736111111115}

In [37]:
returns.median()

AttributeError: 'dict' object has no attribute 'median'

In [ ]:
returns.mean()

riskfree        0.047547
bitcoin         0.066876
gold            0.004493
ftse            0.001722
house_prices    0.004640
bank_rates      0.045877
dtype: float64